### Extract timing files

In [1]:
#r "nuget: System.Data.SQLite, *-*"
#r "nuget: Microsoft.DotNet.Interactive.ExtensionLab, *-*"
#r "nuget: Dapper, *-*"

using System.IO;
using System.Text.Json;
using System.Text.Encodings.Web;
using System.Text.Unicode;
using System.Data.SQLite;
using Dapper;

var jsonSerializerOptions = new JsonSerializerOptions
{
    Encoder = JavaScriptEncoder.Create(UnicodeRanges.All),
    //WriteIndented = true
};

Installed Packages Dapper, 2.1.44 Microsoft.DotNet.Interactive.ExtensionLab, 1.0.0-beta.24229.4 System.Data.SQLite, 1.0.118

Loading extension script from `C:\Users\Faruque\.nuget\packages\microsoft.dotnet.interactive.extensionlab\1.0.0-beta.24229.4\interactive-extensions\dotnet\extension.dib`

Create strongly-typed dataframes using #!linqify . 
 The #!linqify magic command replaces a Microsoft.Data.Analysis.DataFrame variable with a generated, strongly-typed data frame, allowing the use of LINQ operations over the contained data.

ExplainCode 
 This extension generates Sequence diagrams from csharp code using Mermaid kernel.

Inspect code compilation details using the #!inspect magic command. 
 The #!inspect magic command allows you to see the C# decompilation, IL, and JIT Asm for the code in a C# cell.

Use the #!record magic command to keep a transcript of the code you run. 
 Once you enable transcripts using #!record , each code submission (including re-running cells) is recorded in the specified file. The JSON format used is the same format recognized by the .NET Interactive stdio and http APIs and can be used to replay an interactive session via automation.

Loading extensions from `C:\Users\Faruque\.nuget\packages\microsoft.data.analysis\0.21.0\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

#### Recitation Timing (Word by word)

In [3]:
var dbPath = @"D:\DatabaseBackups\QuranDB\recitaion-timings\";

record WbwRecitationTiming(long sura, long ayah, long time, string words);

bool SaveAsJson(string dbFilename){
    var dbFileFullName = dbPath + dbFilename + ".db";
    
    if (!File.Exists(dbFileFullName)) 
        return false;

    using (var connenction = new SQLiteConnection($"Data Source={dbFileFullName};Mode=Memory;Cache=Shared"))
    {
        var result = connenction.Query<WbwRecitationTiming>("SELECT sura, ayah, time, words FROM timings ORDER BY sura, ayah");

        var arrayList = new List<object>();

        foreach (var item in result){
            var words = item.words.Split(',');
            var wordTimes = words.Select(w=>w.Split(':').Select(s=>Convert.ToInt32(s)));
            arrayList.Add(new object[]{ item.sura, item.ayah, item.time, wordTimes, "newLine" });
        }

        var resultJson = JsonSerializer.Serialize(arrayList, jsonSerializerOptions)
                            .Replace(",\"newLine\"],", "]," + Environment.NewLine)
                            .Replace(",\"newLine\"","");
    
        File.WriteAllText(dbPath + "extractedJson\\" + dbFilename + ".json", resultJson);
    }

    return true;
}

SaveAsJson("mishari_alafasy");

#### Recitation Timing (Ayat by Ayat)

In [5]:
var dbPath = @"D:\DatabaseBackups\QuranDB\recitaion-timings\";

record RecitationTiming(Int32 sura, Int32 ayah, Int32 time);

bool SaveAsJson(string dbFilename){
    var dbFileFullName = dbPath + dbFilename + ".db";
    
    if (!File.Exists(dbFileFullName)) 
        return false;

    using (var connenction = new SQLiteConnection($"Data Source={dbFileFullName};Mode=Memory;Cache=Shared"))
    {
        var result = connenction.Query<RecitationTiming>("SELECT sura, ayah, time FROM timings ORDER BY sura, ayah");

        var arrayList = new List<object>();

        foreach (var item in result){       
            if (item.ayah != 999)     
                arrayList.Add(new object[]{ item.sura, item.ayah, item.time, "newLine" });
        }

        var resultJson = JsonSerializer.Serialize(arrayList, jsonSerializerOptions)
                            .Replace(",\"newLine\"],", "]," + Environment.NewLine)
                            .Replace(",\"newLine\"","");
    
        File.WriteAllText(dbPath + "extractedJson\\" + dbFilename + ".json", resultJson);
    }

    return true;
}

SaveAsJson("muaiqly_kfgqpc");

### Download timing files form qurancdn.com

In [2]:
#r "nuget: System.Net.Http.Json, *-*"

Installed Packages System.Net.Http.Json, 9.0.0-preview.5.24306.7

In [3]:
using System.IO;
using System.Text.Json;
using System.Net.Http;
using System.Net.Http.Json;
using System.Text.Encodings.Web;
using System.Text.Unicode;

var jsonSerializerOptions = new JsonSerializerOptions
{
    Encoder = JavaScriptEncoder.Create(UnicodeRanges.All),
    WriteIndented = true
};

var srcPath = @"D:\Faruque\Projects\quranpwa\src\assets\";
var quranDataJson = File.ReadAllText(srcPath + "quran-data.json");

public class QuranData
{
    public List<List<object>> suras { get; set; }
    public List<List<int>> hizb_quarters { get; set; }
    public List<List<int>> manzils { get; set; }
    public List<List<int>> rukus { get; set; }
    public List<List<int>> pages { get; set; }
    public List<List<object>> sajdas { get; set; }
    public List<List<int>> juzs { get; set; }
}

var quranData = JsonSerializer.Deserialize<QuranData>(quranDataJson);

// Root myDeserializedClass = JsonConvert.DeserializeObject<Root>(myJsonResponse);
public class AudioFile
{
    public int id { get; set; }
    public int chapter_id { get; set; }
    public double file_size { get; set; }
    public string format { get; set; }
    public string audio_url { get; set; }
    public int? duration { get; set; }
    public List<VerseTiming> verse_timings { get; set; }
}

public class Root
{
    public List<AudioFile> audio_files { get; set; }
}

public class VerseTiming
{
    public string verse_key { get; set; }
    public int timestamp_from { get; set; }
    public int timestamp_to { get; set; }
    public int? duration { get; set; }
    public List<List<double>> segments { get; set; }
}

#### Download from the server

In [ ]:
var urlBase = "https://api.qurancdn.com/api/qdc/audio/reciters/161/audio_files?segments=true&chapter=";

var audioTimingFiles = new List<AudioFile>();

for(int i = 0; i < quranData.suras.Count; i++){
    var sura = quranData.suras[i];
    var serial = i + 1;

    using (var httpClient = new HttpClient()) {
        var apiUrl = urlBase + serial;
        var apiResponse = httpClient.GetAsync(apiUrl).Result;
        var apiContent = apiResponse.Content.ReadFromJsonAsync<Root>().Result;
        audioTimingFiles.AddRange(apiContent.audio_files);
    }
}

var resultJson = JsonSerializer.Serialize(audioTimingFiles, jsonSerializerOptions);

var downloadPath = @"D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn";
File.WriteAllText(downloadPath + "\\audioTimingFiles.json", resultJson);

Console.WriteLine($"Download completed");

#### Process from file

In [5]:
var timingJson = File.ReadAllText(@"D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn\abdul-basit-audioTimingFiles.json");
var audioTimingFiles = JsonSerializer.Deserialize<List<AudioFile>>(timingJson);
var downloadPath = @"D:\DatabaseBackups\QuranDB\recitaion-timings\downloaded-from-qurancdn";

In [6]:
public class RecitaionTiming {
    public int sura;
    public int ayat;
    public int time;
    public List<List<double>> wordTimings;
}
var recitaionTimings = new List<RecitaionTiming>();

foreach(var audioTimingFile in audioTimingFiles) {
    recitaionTimings.AddRange(audioTimingFile.verse_timings.Select(verse_timing=> {
        var verse_key = verse_timing.verse_key.Split(':');
        return new RecitaionTiming { 
            sura = int.Parse(verse_key[0]),
            ayat = int.Parse(verse_key[1]),
            time = verse_timing.timestamp_from,
            wordTimings = verse_timing.segments
        };
    }));
}

var arrayList = new List<object>();

foreach (var item in recitaionTimings){
    arrayList.Add(new object[]{ item.sura, item.ayat, item.time, item.wordTimings, "newLine" });
}

var resultJson = JsonSerializer.Serialize(arrayList)
                    .Replace(",\"newLine\"],", "]," + Environment.NewLine)
                    .Replace(",\"newLine\"","");

File.WriteAllText(downloadPath + "\\abdul-basit-audioTimingFiles_transformed.json", resultJson);

audioTimingFiles.Count

114

### Downlaod from quran.gov.bd

In [ ]:
using System.IO;
using System.Text.Json;
using System.Net;

var srcPath = @"D:\Faruque\Projects\quranpwa\src\assets\";
var quranDataJson = File.ReadAllText(srcPath + "quran-data.json");

public class QuranData
{
    public List<List<object>> suras { get; set; }
    public List<List<int>> hizb_quarters { get; set; }
    public List<List<int>> manzils { get; set; }
    public List<List<int>> rukus { get; set; }
    public List<List<int>> pages { get; set; }
    public List<List<object>> sajdas { get; set; }
    public List<List<int>> juzs { get; set; }
}

var quranData = JsonSerializer.Deserialize<QuranData>(quranDataJson);

record DownloadableFile(string url, string filename);

var downlaodableFiles = new List<DownloadableFile>();

for(int i = 0; i < quranData.suras.Count; i++){
    var sura = quranData.suras[i];
    var serial = i + 1;
    //var start = (int)sura[0];
    var ayas = int.Parse(sura[1].ToString());

    for(int j = 1; j <= ayas; j++){
        downlaodableFiles.Add( new($"{serial}/{serial}-{j}.mp3", $"{serial.ToString().PadLeft(3, '0')}{j.ToString().PadLeft(3, '0')}.mp3"));
    }
}

var urlBase = "http://quran.gov.bd/quran/Sound/bangla";
var downloadPath = @"D:\Faruque\Projects\QuranAudio\Mahbub_Subhan_quran.gov.bd";

foreach (var downlaodableFile in downlaodableFiles){
    string url = $"{urlBase}/{downlaodableFile.url}";
    string filePath = $"{downloadPath}\\{downlaodableFile.filename}";

    if (File.Exists(filePath))
        continue;

    using (var client = new WebClient())
    {
        client.DownloadFile(url, filePath);
    }
    Console.WriteLine($"Downloaded file: {downlaodableFile}");
}

Console.WriteLine($"Download completed");
